In [5]:
import numpy as np
from graph_tool.all import Graph, shortest_path
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
from collections import defaultdict
import os
import python_codes.files_operators as fo

# === Step 1: 设置城市与规模 ===
city = "2keihan2"
scale = "500"
filename = f"./MST_net/{city}_pop_{scale}_mst.net"

# === Step 2: 读取图与节点位置 ===
read_graph, read_pos = fo.read_files(filename)

# === Step 3: 计算质心 ===
positions = np.array([np.array(read_pos[v]) for v in read_graph.vertices()])
centroid = np.mean(positions, axis=0)

# === Step 4: 获取叶节点 ===
def get_leaf_nodes(g):
    return [int(v) for v in g.vertices() if g.vertex(v).out_degree() == 1]

leaf_nodes = get_leaf_nodes(read_graph)

# === Step 5: 距离等间隔分层 ===
def assign_layers_by_distance(pos, leaf_nodes, center, num_layers):
    dist_list = [(v, np.linalg.norm(np.array(pos[v]) - center)) for v in leaf_nodes]
    dists = [d for _, d in dist_list]
    d_min, d_max = min(dists), max(dists)
    interval = (d_max - d_min) / num_layers

    layers = defaultdict(list)
    for v, d in dist_list:
        layer = int((d - d_min) / interval)
        if layer == num_layers:
            layer -= 1
        layers[layer].append(v)
    return layers

# === Step 6: 构建环连接 ===
def construct_rings_by_angle(pos, layers, center):
    ring_edges = []
    edge_layer_map = {}
    for layer, nodes in layers.items():
        if len(nodes) <= 2:
            continue
        sorted_nodes = sorted(nodes, key=lambda v: np.arctan2(pos[v][1] - center[1], pos[v][0] - center[0]))
        for i in range(len(sorted_nodes)):
            u, v = sorted_nodes[i], sorted_nodes[(i + 1) % len(sorted_nodes)]
            ring_edges.append((u, v))
            edge_layer_map[(u, v)] = layer
    return ring_edges, edge_layer_map

# === Step 7: 层颜色 ===
def get_layer_colors():
    return {
        0: 'red',
        1: 'orange',
        2: 'yellow',
        3: 'green',
        4: 'blue',
        5: 'purple',
        6: 'brown'
    }

# === Step 8: 绘图函数（环数量 + 半径标注）===
def draw_graph_with_radius_info(g, pos, added_edges, edge_layer_map, leaf_nodes, save_path, centroid=None, layers=None):
    fig, ax = plt.subplots(figsize=(8, 8))

    # 原始 MST 边
    for e in g.edges():
        s, t = int(e.source()), int(e.target())
        x = [pos[s][0], pos[t][0]]
        y = [pos[s][1], pos[t][1]]
        ax.plot(x, y, color='gray', linewidth=0.6, zorder=1)

    # 新连接边
    layer_colors = get_layer_colors()
    for (u, v) in added_edges:
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        layer = edge_layer_map.get((u, v), edge_layer_map.get((v, u), 0))
        ax.plot(x, y, color=layer_colors.get(layer, 'red'), linewidth=1.0, zorder=2)

    # 节点绘制
    for v in g.vertices():
        x, y = pos[int(v)][0], pos[int(v)][1]
        if int(v) in leaf_nodes:
            ax.scatter(x, y, color='red', s=8, zorder=3)
        else:
            ax.scatter(x, y, color='gray', s=8, zorder=2)

    # 中心点
    if centroid is not None:
        ax.scatter(centroid[0], centroid[1], color='lime', marker='x', s=60, zorder=4)

    # 辅助圆 & 标注
    if layers is not None:
        for layer, nodes in layers.items():
            if not nodes:
                continue
            dists = [np.linalg.norm(np.array(pos[v]) - centroid) for v in nodes]
            avg_radius = np.mean(dists)
            circle = Circle(centroid, avg_radius, edgecolor='lightgreen',
                            linestyle='--', linewidth=1.0, fill=False, zorder=0)
            ax.add_patch(circle)

            # 文本标注（环编号 + 平均半径 + 节点数）
            angle = np.pi / 4
            label_x = centroid[0] + avg_radius * np.cos(angle)
            label_y = centroid[1] + avg_radius * np.sin(angle)
            text = f"L{layer} / R={avg_radius:.1f} / {len(nodes)}"
            ax.text(label_x, label_y, text, fontsize=8,
                    color=layer_colors.get(layer, 'black'), ha='center')

    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# === Step 9: 拓扑评估 ===
def evaluate_congestion(graph):
    edge_usage = defaultdict(int)
    total_path_length = 0
    for v1 in graph.vertices():
        for v2 in graph.vertices():
            if int(v1) >= int(v2):
                continue
            path = shortest_path(graph, v1, v2)[1]
            total_path_length += len(path)
            for e in path:
                s, t = int(e.source()), int(e.target())
                edge = tuple(sorted((s, t)))
                edge_usage[edge] += 1
    congestion_cost = sum(edge_usage.values())
    return total_path_length, congestion_cost

# === Step 10: 主流程（环数：2~4，可扩展）===
layer_options = list(range(2, 8))
results = []

output_dir = f"./output/rings_by_equal_distance/{city}_pop_{scale}_mst"
os.makedirs(output_dir, exist_ok=True)

for num_layers in layer_options:
    layers = assign_layers_by_distance(read_pos, leaf_nodes, centroid, num_layers)
    ring_edges, edge_layer_map = construct_rings_by_angle(read_pos, layers, centroid)
    new_g = Graph(read_graph)
    for u, v in ring_edges:
        new_g.add_edge(new_g.vertex(u), new_g.vertex(v))

    total_len, congestion = evaluate_congestion(new_g)
    results.append((num_layers, total_len, congestion))

    save_path = f"{output_dir}/{city}_mst_ring_L{num_layers}.png"
    draw_graph_with_radius_info(new_g, read_pos, ring_edges, edge_layer_map, leaf_nodes,
                                save_path, centroid=centroid, layers=layers)

# === Step 11: 保存 CSV 表格 ===
df = pd.DataFrame(results, columns=["层数", "最短路径总和", "拥堵代价"])
df.to_csv(f"{output_dir}/{city}_ring_results.csv", index=False)

print(f"✅ 图与表保存完成：{output_dir}/")


✅ 图与表保存完成：./output/rings_by_equal_distance/2keihan2_pop_500_mst/
